# Decision Trees und Random Forests

## Quelle der Daten

https://www.kaggle.com/datasets/captainozlem/framingham-chd-preprocessed-data (zuletzt aufgerufen: 01/2024)

https://www.framinghamheartstudy.org/ (zuletzt aufgerufen: 01/2024)

## Installation der Bibliotheken

In [ ]:
# Importieren der benötigten Bibliotheken
import pandas as pd # Für die Arbeit mit Tabellen/DataFrames

import matplotlib.pyplot as plt # Für die Erstellung von Diagrammen und Visualisierungen

from imblearn.over_sampling import SMOTE # Für das Resampling unausgeglichener Datensätze

from sklearn.model_selection import train_test_split # Für die Aufteilung von Daten in Trainings- und Testsets

# Importiert die DecisionTreeClassifier-Klasse aus dem Modul 'tree' der Bibliothek 'sklearn'.
# Diese Klasse wird verwendet, um ein Entscheidungsbaummodell zu erstellen.
from sklearn.tree import DecisionTreeClassifier

# Importiert die RandomForestClassifier-Klasse aus dem Modul 'ensemble' der Bibliothek 'sklearn'.
# Diese Klasse wird verwendet, um ein Random-Forest-Modell zu erstellen, das aus mehreren Entscheidungsbäumen besteht.
from sklearn.ensemble import RandomForestClassifier

# Importiert das Modul 'tree' aus der Bibliothek 'sklearn'.
# Dieses Modul enthält Funktionen zur Visualisierung von Entscheidungsbäumen.
from sklearn import tree


## Einlesen der Daten

In [ ]:
# Definieren der URL der Datenquelle und Laden der Daten in einen pandas DataFrame
data_url = "https://github.com/timwgnd/Lehrbuch-Kuenstliche-Intelligenz-in-der-Medizin/raw/refs/heads/main/FraminghamHeartStudy.xlsx"
data = pd.read_excel(io=data_url, sheet_name = "Tabelle1")

# Entfernen von Zeilen mit fehlenden Werten
data = data.dropna()

# Anzeigen der ersten Zeilen des DataFrames, um einen Überblick über die Daten zu erhalten
print(data.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Erstellen von Dummy-Variablen für die kategoriale Spalte "Geschlecht"
# Dies wandelt die kategoriale Variable in numerische Variablen um, die für das Modell verwendet werden können
data_new = pd.get_dummies(data, columns = ["Geschlecht"])

# Anzeigen der ersten Zeilen des neuen DataFrames mit den Dummy-Variablen
print(data_new.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Zählen der Häufigkeit der einzelnen Werte in der Spalte "Diabetes"
count_diabetes = data_new["Diabetes"].value_counts()

# Anzeigen der Häufigkeitsauzszählung in Tabellenform
print(count_diabetes.to_markdown(tablefmt='psql'))

# Erstellen eines Balkendiagramms, das die Verteilung der Werte in der Spalte "Diabetes" zeigt
count_diabetes.plot(kind = "bar", rot = 0)

## Aufteilung der Daten und Resampling

In [ ]:
# Wählt die Spalten mit den Indizes 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13 und 14 aus dem DataFrame 'data_new' aus.
# Diese Spalten werden als unabhängige Variablen (Features) für das Entscheidungsbaummodell verwendet.
x = data_new.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]]

# Wählt die Spalte mit dem Index 7 ('Diabetes') aus dem DataFrame 'data_new' aus.
# Diese Spalte wird als abhängige Variable (Zielvariable) für das Entscheidungsbaummodell verwendet.
y = data_new.iloc[:, 7]

In [ ]:
# Anzeigen der Feature-Variablen x
print(x.head().to_markdown(index=False, tablefmt='psql'))

In [ ]:
# Anzeigen der Zielvariable y
print(y)

In [ ]:
# Aufteilen der Daten in Trainings- und Testsets
# test_size = 0.15 bedeutet, dass 15% der Daten für das Testset verwendet werden
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15)

In [ ]:
# Initialisieren des SMOTE-Objekts für Over-Sampling
sm = SMOTE()

# Anwenden von SMOTE auf die Trainingsdaten, um die Anzahl der Instanzen der Minderheitenklasse zu erhöhen
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

# Anzeigen der Verteilung der Zielvariablen im resampelten Trainingsset, um den Effekt von SMOTE zu sehen
print(y_train_res.value_counts().to_markdown(tablefmt='psql'))

## Decision-Tree-Modell

In [ ]:
# Erstellt ein DecisionTreeClassifier-Objekt (Entscheidungsbaum-Klassifikator) mit dem 
# Kriterium "Entropie", einer maximalen Tiefe von 2 und keiner Begrenzung der Blattknoten.
dt = DecisionTreeClassifier(criterion = "entropy", max_depth = 2, max_leaf_nodes = None)

# Trainiert das Entscheidungsbaummodell mit den resampleten Trainingsdaten (x_train_res, y_train_res).
dt.fit(x_train_res, y_train_res)

# Berechnet die Genauigkeit (accuracy) des trainierten Entscheidungsbaummodells auf den Testdaten (x_test, y_test).
dt.score(x_test, y_test)


In [ ]:
# Definiert eine Liste von Merkmalen (Features), die für die Vorhersage verwendet werden.
# Diese Liste enthält Informationen wie Alter, Raucherstatus, Anzahl der Zigaretten pro Tag usw.
characteristics = ["Alter", "Raucher", "ZigProTag", "BDMedikamente", "Schlaganfall", 
                   "ArtHypertonie", "GesCholesterin", "Systolisch", "Diastolisch", 
                   "BMI", "Herzfrequenz", "Blutzucker", "Geschlecht_M"]

# Definiert eine Liste möglicher Diagnosen.
# "0" steht für "kein Diabetes" und "1" für "Diabetes vorhanden".
diagnosis = ["0" , "1"]

# Erstellt ein Diagramm des Entscheidungsbaums (dt).
# 'feature_names' legt die Namen der Merkmale fest, die in den Knoten des Baums angezeigt werden.
# 'class_names' legt die Namen der Klassen fest, die in den Blättern des Baums angezeigt werden.
tree.plot_tree(dt, feature_names = characteristics, class_names = diagnosis)


In [ ]:
# Erstellt ein DecisionTreeClassifier-Objekt (Entscheidungsbaum-Klassifikator) mit dem 
# Kriterium "Entropie", einer maximalen Tiefe von 5 und keiner Begrenzung der Blattknoten.
dt = DecisionTreeClassifier(criterion = "entropy", max_depth = 5, max_leaf_nodes = None)

# Trainiert das Entscheidungsbaummodell (dt) mit den resampleten Trainingsdaten (x_train_res, y_train_res).
dt.fit(x_train_res, y_train_res)

# Berechnet die Genauigkeit (accuracy) des trainierten Entscheidungsbaummodells auf den Testdaten (x_test, y_test).
dt.score(x_test, y_test)

## Random-Forest-Modell

In [ ]:
# Erstellt ein RandomForestClassifier-Objekt (Random Forest Klassifikator) mit dem
# Gini-Kriterium, einer maximalen Tiefe von 5 und keiner Begrenzung der Blattknoten.
rf = RandomForestClassifier(criterion = "gini", max_depth = 5, max_leaf_nodes = None)

# Trainiert das Random-Forest-Modell (rf) mit den resampleten Trainingsdaten (x_train_res, y_train_res).
rf.fit(x_train_res, y_train_res)

# Berechnet die Genauigkeit (accuracy) des trainierten Random-Forest-Modells auf den Testdaten (x_test, y_test).
rf.score(x_test, y_test)
